In [1]:
!pip install flask flask_cors requests pyngrok

In [2]:
!ngrok config add-authtoken "ngrok token"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import requests
from pyngrok import ngrok

# 🔑 Your Gemini API Key
GEMINI_API_KEY = "Your_API_Key"

app = Flask(__name__)
CORS(app)



In [4]:
# ⭐ Gemini moderation function
import json
import re
import requests

def moderate_with_gemini(text):
    url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?key={"Your_API_Key"}"

    prompt = f"""
You are an AI moderation model.

Classify the given text into ONLY one category:
- safe
- toxic
- hate
- violence
- self_harm
- spam

Return ONLY valid JSON in this exact format:

{{
  "category": "...",
  "reason": "...",
  "action": "..."
}}

User text: {text}

Rules:
- Do NOT add explanation or markdown.
- Return ONLY raw JSON.
"""

    # Send request
    payload = {
        "contents": [
            {"parts": [{"text": prompt}]}
        ]
    }

    response = requests.post(url, json=payload)

    # If API error
    if response.status_code != 200:
        return {
            "error": "API error",
            "code": response.status_code,
            "details": response.text
        }

    # Extract model response
    try:
        data = response.json()
        raw_text = data["candidates"][0]["content"]["parts"][0]["text"]
    except:
        return {
            "error": "Parsing error",
            "details": "Could not extract text",
            "raw": response.text
        }

    # Clean response
    cleaned = raw_text.strip()
    cleaned = cleaned.replace("\\n", "\n")

    # 🔍 Find JSON using simple safe regex
    match = re.search(r"\{.*?\}", cleaned, flags=re.DOTALL)

    if not match:
        return {
            "error": "Parsing error",
            "details": "No JSON object found",
            "raw": cleaned
        }

    json_str = match.group(0).strip()

    # Parse extracted JSON
    try:
        parsed = json.loads(json_str)
        return parsed
    except Exception as e:
        return {
            "error": "Invalid JSON",
            "details": str(e),
            "extract": json_str,
            "raw": cleaned
        }


In [5]:

# ⭐ API for your UI
@app.route('/moderate', methods=['POST'])
def moderate_api():
    data = request.json
    text = data.get("text", "")

    result = moderate_with_gemini(text)
    print("DeBUg ReSULT ->",result)
    return jsonify({"moderation_result": result})



In [6]:
moderate_with_gemini("I hate you")

{'error': 'API error',
 'code': 400,
 'details': '{\n  "error": {\n    "code": 400,\n    "message": "API key not valid. Please pass a valid API key.",\n    "status": "INVALID_ARGUMENT",\n    "details": [\n      {\n        "@type": "type.googleapis.com/google.rpc.ErrorInfo",\n        "reason": "API_KEY_INVALID",\n        "domain": "googleapis.com",\n        "metadata": {\n          "service": "generativelanguage.googleapis.com"\n        }\n      },\n      {\n        "@type": "type.googleapis.com/google.rpc.LocalizedMessage",\n        "locale": "en-US",\n        "message": "API key not valid. Please pass a valid API key."\n      }\n    ]\n  }\n}\n'}

In [7]:
# 🚀 Start Flask + ngrok
public_url = ngrok.connect(5000)
print("🔥 NGROK URL:", public_url)
app.run(port=5000)


ERROR:pyngrok.process.ngrok:t=2025-11-27T06:02:23+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: ngrok token\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"
ERROR:pyngrok.process.ngrok:t=2025-11-27T06:02:23+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: ngrok token\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n"


PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified does not look like a proper ngrok authtoken.\nYour authtoken: ngrok token\nInstructions to install your authtoken are on your ngrok dashboard:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_105\r\n.